In [1]:
import sys
sys.path.append("../")

import pandas as pd
import seaborn as sns
import numpy as np

sns.set_style('darkgrid')
sns.set(font_scale = 1.5)

In [2]:
from src.finman.utils.gsheet import GSheetWorker

In [3]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

gsw = GSheetWorker(gcreds_file)

# 1. Data Processing

In [4]:
sheet_setups = [
    {
        "bank_sheet_name": "t-bank (main)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "t-bank (deposit)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "t-bank (invest)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции в валюте карты",
    },
    {
        "bank_sheet_name": "ozon-bank (main)",
        "dt_col": "Время операции",
        "trans_rub_col": "Сумма операции",
    },
]

transaction_lists = [
    "t-bank (main)", "ozon-bank (main)"
]

cat_col = "Категория"
subcat_col = "Подкатегория"
debt_col = "Долг"

In [5]:
trans_df = []
for setup in sheet_setups:
    gs_df = gsw.get_df(sheet_id, setup["bank_sheet_name"])

    gs_df = gs_df.rename(columns={
        setup['dt_col']: "dt",
        setup['trans_rub_col']: "trans_rub",

    })

    if setup["bank_sheet_name"] in transaction_lists:
        trans_df.append(gs_df[[
            "dt", "trans_rub", cat_col, subcat_col, debt_col
        ]])
    else:
        trans_df.append(gs_df[[
            "dt", "trans_rub", cat_col, subcat_col
        ]])

    trans_df[-1]['account'] = setup["bank_sheet_name"]

    print(setup["bank_sheet_name"] + " done!")

trans_df = pd.concat(trans_df)

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_41888/27192228.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df[-1]['account'] = setup["bank_sheet_name"]


t-bank (main) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_41888/27192228.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df[-1]['account'] = setup["bank_sheet_name"]


t-bank (deposit) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_41888/27192228.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df[-1]['account'] = setup["bank_sheet_name"]


t-bank (invest) done!
ozon-bank (main) done!


/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_41888/27192228.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df[-1]['account'] = setup["bank_sheet_name"]


In [6]:
trans_df.head()

,dt,trans_rub,Категория,Подкатегория,Долг,account
0,2024-08-19 12:45,-725.85,Food & Dining,Business Lunch,None,t-bank (main)
1,2024-08-19 9:06,-1103,Food & Dining,Delivery,None,t-bank (main)
2,2024-08-18 19:46,-197,Transportation,Carsharing,None,t-bank (main)
3,2024-08-18 13:16,-640,Household,Groceries,,t-bank (main)
4,2024-08-18 13:12,-580,Household,Groceries,None,t-bank (main)


In [7]:
trans_df['dt'] = pd.to_datetime(trans_df['dt'])
trans_df['trans_rub'] = pd.to_numeric(trans_df['trans_rub'])

In [8]:
exp_mask = (trans_df['trans_rub'] < 0) & (trans_df['account'].isin(transaction_lists) & (trans_df['Категория'] != "Transfer"))
inc_mask = (trans_df['trans_rub'] > 0)

exp_df = trans_df.loc[exp_mask]
exp_df['trans_rub'] = exp_df['trans_rub'].abs()

/var/folders/n9/7qzl9sfj3_n9hmnht95_9jy80000gn/T/ipykernel_41888/320991895.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['trans_rub'] = exp_df['trans_rub'].abs()


In [9]:
months_dict = {key: value for key,value in zip(np.arange(1, 13), ["январь", "февраль", "март", "апрель", "май", "июнь", 
                                                                    "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"])}

In [10]:
months_dict

{1: 'январь',
 2: 'февраль',
 3: 'март',
 4: 'апрель',
 5: 'май',
 6: 'июнь',
 7: 'июль',
 8: 'август',
 9: 'сентябрь',
 10: 'октябрь',
 11: 'ноябрь',
 12: 'декабрь'}

# Пончик трат по категориям

In [11]:
month = 8

In [12]:
df_vis = exp_df.loc[exp_df['dt'].dt.month == month]

In [16]:
import plotly.graph_objects as go

# Function to plot the main category pie chart
def plot_category_pie(df):
    # Grouping data by category
    category_data = df.groupby('Категория')[['trans_rub']].sum()
    
    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=category_data.index, values=category_data.values, 
                                 hole=0.3)])

    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15,
                      pull=[0.1 if category_data.index[i] == 'Food' else 0 for i in range(len(category_data))])

    # Adding callback function for click event
    fig.update_layout(title_text="Transaction Distribution by Category")
    fig.data[0].on_click(lambda trace, points, state: plot_subcategory_pie(df, points))

    return fig

# Function to plot the subcategory pie chart
def plot_subcategory_pie(df, points):
    category_selected = points.point_inds[0]
    category_name = df['Категория'].unique()[category_selected]
    
    # Filter data based on selected category
    subcategory_data = df[df['category'] == category_name].groupby('subcategory').sum()['amount']

    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=subcategory_data.index, values=subcategory_data.values, 
                                 hole=0.3)])
    
    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15,
                      pull=[0.1 if subcategory_data.index[i] == 'Dining Out' else 0 for i in range(len(subcategory_data))])
    
    # Adding callback function for click event
    fig.update_layout(title_text=f"Transaction Distribution for {category_name} by Subcategory")
    fig.data[0].on_click(lambda trace, points, state: plot_category_pie(df))
    
    fig.show()

In [17]:
fig = plot_category_pie(df_vis)
fig.show()

In [21]:
import plotly.graph_objects as go
import pandas as pd

# Sample data frame
data = {
    'date_time': ['2024-01-01 10:00:00', '2024-01-01 11:00:00', '2024-01-02 12:00:00',
                  '2024-01-02 13:00:00', '2024-01-03 14:00:00', '2024-01-03 15:00:00'],
    'amount': [100, 150, 200, 130, 120, 110],
    'category': ['Food', 'Transport', 'Food', 'Entertainment', 'Transport', 'Food'],
    'subcategory': ['Groceries', 'Bus', 'Dining Out', 'Movies', 'Train', 'Dining Out']
}

df = pd.DataFrame(data)

# Function to plot the main category pie chart
def plot_category_pie(df):
    # Grouping data by category
    category_data = df.groupby('category').sum()['amount']
    
    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=category_data.index, values=category_data.values, 
                                 hole=0.3)])

    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15)

    # Adding callback function for click event
    fig.update_layout(title_text="Transaction Distribution by Category")
    fig.data[0].on_click(lambda trace, points, state: plot_subcategory_pie(df, points))

    return fig

# Function to plot the subcategory pie chart
def plot_subcategory_pie(df, points):
    category_selected = points.point_inds[0]
    category_name = df['category'].unique()[category_selected]
    
    # Filter data based on selected category
    subcategory_data = df[df['category'] == category_name].groupby('subcategory').sum()['amount']

    # Creating the pie chart
    fig = go.Figure(data=[go.Pie(labels=subcategory_data.index, values=subcategory_data.values, 
                                 hole=0.3)])
    
    fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', 
                      textfont_size=15,
                      pull=[0.1 if subcategory_data.index[i] == 'Dining Out' else 0 for i in range(len(subcategory_data))])
    
    # Adding callback function for click event
    fig.update_layout(title_text=f"Transaction Distribution for {category_name} by Subcategory")
    fig.data[0].on_click(lambda trace, points, state: plot_category_pie(df))
    
    fig.show()

# Plotting the initial pie chart
fig = plot_category_pie(df)
fig.show()


In [20]:
import plotly.graph_objects as go
import pandas as pd

# Sample data frame (replace with your actual data)
data = {
    'date_time': ['2024-01-01 10:00:00', '2024-01-01 11:00:00', '2024-01-02 12:00:00',
                  '2024-01-02 13:00:00', '2024-01-03 14:00:00', '2024-01-03 15:00:00'],
    'amount': [100, 150, 200, 130, 120, 110],
    'category': ['Food', 'Transport', 'Food', 'Entertainment', 'Transport', 'Food'],
    'subcategory': ['Groceries', 'Bus', 'Dining Out', 'Movies', 'Train', 'Dining Out']
}

df = pd.DataFrame(data)

# Grouping data by category and subcategory
category_data = df.groupby('category').sum()['amount']
subcategory_data = {category: df[df['category'] == category].groupby('subcategory').sum()['amount']
                    for category in df['category'].unique()}

# Create initial pie chart
fig = go.Figure()

# Add traces for each category
fig.add_trace(go.Pie(labels=category_data.index, values=category_data.values, hole=0.3, name="Category"))

# Add traces for each subcategory, initially hidden
for category, sub_data in subcategory_data.items():
    fig.add_trace(go.Pie(labels=sub_data.index, values=sub_data.values, hole=0.3, visible=False, name=f"Subcategory of {category}"))

# Update layout to include dropdown for interaction
dropdown_buttons = [
    {'label': 'Category View', 'method': 'update', 'args': [{'visible': [True] + [False] * len(subcategory_data)}, {'title': 'Transaction Distribution by Category'}]},
]

# Add buttons for each category to switch to subcategory view
for i, category in enumerate(subcategory_data.keys()):
    visibility = [False] * (1 + len(subcategory_data))
    visibility[i + 1] = True
    dropdown_buttons.append(
        {'label': f'Subcategory of {category}', 'method': 'update', 'args': [{'visible': visibility}, {'title': f'Transaction Distribution for {category} by Subcategory'}]}
    )

# Apply the dropdown menu to the layout
fig.update_layout(
    updatemenus=[
        {
            'active': 0,
            'buttons': dropdown_buttons,
        }
    ]
)

fig.update_layout(title_text="Transaction Distribution by Category")

fig.show()


In [27]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

# Sample DataFrame creation
data = {
    'datetime': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'amount': [100, 150, 200, 300, 450, 600, 120, 330, 220, 500] * 10,
    'category': ['Food', 'Transport', 'Utilities', 'Entertainment', 'Health'] * 20,
    'subcategory': ['Groceries', 'Bus', 'Electricity', 'Movies', 'Doctor'] * 20
}

df = pd.DataFrame(data)

# Function to create a pie chart based on category distribution
def create_category_pie(df):
    category_dist = df.groupby('category')['amount'].sum().reset_index()
    fig = px.pie(category_dist, values='amount', names='category',
                 title='Transaction Distribution by Category',
                 hover_data=['amount'], labels={'amount':'Total Amount'})

    # Adding customdata to pass category info
    fig.update_traces(customdata=category_dist['category'], 
                      hovertemplate='Category: %{customdata}<br>Amount: %{value}<extra></extra>')

    return fig

# Function to create a pie chart for a specific category's subcategories
def create_subcategory_pie(df, category):
    subcategory_dist = df[df['category'] == category].groupby('subcategory')['amount'].sum().reset_index()
    fig = px.pie(subcategory_dist, values='amount', names='subcategory',
                 title=f'Transaction Distribution in {category}',
                 hover_data=['amount'], labels={'amount':'Total Amount'})

    # Adding customdata to allow returning to category view
    fig.update_traces(customdata=[None]*len(subcategory_dist), 
                      hovertemplate='Subcategory: %{label}<br>Amount: %{value}<extra></extra>')

    return fig

# Initial pie chart
category_fig = create_category_pie(df)
category_fig.show()

# For interactive handling in Jupyter notebook
output = widgets.Output()

def handle_click(trace, points, state):
    with output:
        output.clear_output()
        if points.points[0].customdata is not None:  # Drill down
            category = points.points[0].customdata
            subcategory_fig = create_subcategory_pie(df, category)
            subcategory_fig.show()
        else:  # Go back
            category_fig.show()

# Connect the callback to the figure
category_fig.data[0].on_click(handle_click)

# Display the initial chart
with output:
    category_fig.show()

display(output)


Output()

ERROR: Could not find a version that satisfies the requirement ipywidget (from versions: none)
ERROR: No matching distribution found for ipywidget
You should consider upgrading via the '/Users/apenkin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [66]:
import plotly.express as px

# fig = px.pie(trans_df.loc[exp_mask & (trans_df['dt'].dt.month == month)], values='trans_rub', names='Категория', title=f'Траты за месяц {month}')
# df_tmp = trans_df.loc[exp_mask]

fig = px.pie(df_vis, values='trans_rub', names='Категория', title=f'Всего трат за {months_dict[month]}: {df_vis["trans_rub"].sum()}')
fig.show()

In [69]:
df_vis_2 = df_vis.loc[df_vis['Подкатегория'] != "Mortgage/Rent"]

fig = px.pie(df_vis_2, values='trans_rub', names='Категория', title=f'Всего трат за {months_dict[month]}: {df_vis_2["trans_rub"].sum()}')
fig.show()

In [43]:
trans_df

,dt,trans_rub,Категория,Подкатегория,Долг,account
0,2024-08-19 12:45:00,-725.85,Food & Dining,Business Lunch,None,t-bank (main)
1,2024-08-19 09:06:00,-1103.00,Food & Dining,Delivery,None,t-bank (main)
2,2024-08-18 19:46:00,-197.00,Transportation,Carsharing,None,t-bank (main)
3,2024-08-18 13:16:00,-640.00,Household,Groceries,,t-bank (main)
4,2024-08-18 13:12:00,-580.00,Household,Groceries,None,t-bank (main)
...,...,...,...,...,...,...
42,2024-04-16 16:56:00,-1495.00,Transfer,None,None,ozon-bank (main)
43,2024-04-15 22:38:00,-1302.00,Household,Accessories,None,ozon-bank (main)
44,2024-04-15 08:45:00,-2363.00,Household,Accessories,None,ozon-bank (main)
45,2024-04-13 15:32:00,-4840.00,Household,Accessories,None,ozon-bank (main)
